In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
import numpy as np
import os, json, cv2, random
import cv2


In [ ]:
from livecell_tracker.sample_data import tutorial_three_image_sys
from pathlib import Path
from livecell_tracker.core.datasets import LiveCellImageDataset
import glob


In [ ]:
import napari
napari.__version__

load and save data to a json file containing single cell objects

EBSS data, 24hours

In [ ]:

# dataset_dir_path = Path(
#     "../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/XY16/"
# )

# mask_dataset_path = Path("../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/XY16/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# from livecell_tracker.core import SingleCellStatic
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_scs_EBSS_starvation/single_cells.json",
#                                          dataset_dir="./datasets/test_scs_EBSS_starvation/datasets")


EBSS data, 5 days

In [ ]:

# dataset_dir_path = Path(
#     "../datasets/EBSS_Starvation/tifs_STAV-A549_VIM_5days_EBSS_NA_YL_Ti2e_2022-12-21001/XY3/"
# )

# mask_dataset_path = Path("../datasets/EBSS_Starvation/tifs_STAV-A549_VIM_5days_EBSS_NA_YL_Ti2e_2022-12-21001/out/XY3/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# from livecell_tracker.core import SingleCellStatic
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_scs_EBSS_starvation/single_cells.json",
#                                          dataset_dir="./datasets/test_scs_EBSS_starvation/datasets")


In [ ]:


# dic_dataset, mask_dataset = tutorial_three_image_sys()


In [ ]:
# from skimage.measure import regionprops
# from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)
# SingleCellStatic.write_single_cells_json(single_cells, 
#                                          path="./datasets/test_data_STAV-A549/tutorial_three_img_single_cells.json",
#                                          dataset_dir="./datasets/test_data_STAV-A549/datasets")

In [ ]:
# scs_path = "./datasets/test_data_STAV-A549/tutorial_three_img_single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/single_cells.json"
# scs_path = "./datasets/test_scs_EBSS_starvation/XY16/tmp_corrected_scs.json"
scs_path = "./datasets/test_scs_EBSS_starvation/XY1/single_cells.json"
single_cells = SingleCellStatic.load_single_cells_json(path=scs_path)

In [ ]:
dic_dataset = single_cells[0].img_dataset

In [ ]:
from livecell_tracker.core.sc_seg_operator import ScSegOperator, create_sc_seg_napari_ui
# replace the path below with your downdloaded CSN checkpoint path
ckpt_path = r"./notebook_results/csn_models/model_v11_epoch=3282-test_loss=2.3688.ckpt"
ScSegOperator.load_default_csn_model(path=ckpt_path, cuda=True);

In [ ]:
def enable_gui_qt():
    from IPython import get_ipython

    ipython = get_ipython()
    ipython.run_line_magic('gui', 'qt')


enable_gui_qt()

In [ ]:
from livecell_tracker.core.sct_operator import create_scs_edit_viewer
# sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset)

In [ ]:
single_cells = single_cells


# sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, viewer = viewer, time_span=(0, span_interval))

def annotate_by_interval(single_cells, span_interval = 10, viewer = None, clear_prev_batch=True):
    # TODO: a potential bug is that the slice index is not the same concept as the time. A solution is to add time frame to shape properties
    # Here for now we assume indices represents timeframes 
    sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, viewer = viewer, time_span=(0, span_interval))
    viewer = sct_operator.viewer
    tmp_points_layer_key = '_lcx_sct_cur_idx'
    if "cur_idx" not in viewer.layers:
        points = np.zeros((1, 3))
        points_layer = viewer.add_points(points, name=tmp_points_layer_key)
        points_layer.metadata['cur_idx'] = 0
    else:
        points_layer = viewer.layers['cur_idx']
    viewer.dims.set_point(0, points_layer.metadata['cur_idx'])
    points_layer.metadata["cur_sct_operator"] = sct_operator

    def move_span(viewer, offset):
        try:
            points_layer = viewer.layers[tmp_points_layer_key]
            sct_operator = points_layer.metadata["cur_sct_operator"]
            cur_idx = points_layer.metadata['cur_idx']
            if not (sct_operator is None):
                sct_operator_scs = sct_operator.get_all_scs()
                all_sc_set = set(single_cells)
                for sc in sct_operator_scs:
                    if sc not in all_sc_set:
                        print("<add new sc>:", sc)
                        single_cells.append(sc)
                for sc in single_cells:
                    if sc.timeframe < cur_idx or sc.timeframe > cur_idx+span_interval:
                        continue
                    if sc not in sct_operator_scs:
                        print("<remove sc>:", sc)
                        single_cells.remove(sc)

            
            cur_idx += offset
            cur_idx = min(cur_idx, len(single_cells)-span_interval)
            cur_idx = max(cur_idx, 0)
            points_layer.metadata['cur_idx'] = cur_idx
            cur_span = (cur_idx, cur_idx+span_interval)
            if clear_prev_batch:
                sct_operator.close()
            sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, viewer = viewer, time_span=cur_span)
            points_layer.metadata["cur_sct_operator"] = sct_operator
            viewer.dims.set_point(0, cur_idx)
        except Exception as e:
            print("Error:", e)
            print("Failed to load next span. Please try again.")

    @viewer.bind_key('n')
    def next_span(viewer):
        move_span(viewer, span_interval)

    @viewer.bind_key('b')
    def prev_span(viewer):
        move_span(viewer, -span_interval)



annotate_by_interval(single_cells, span_interval = 10, viewer = None)
# for i in range(0, len(single_cells), span_interval):
#     sct_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, viewer = viewer, time_span=(i, i+span_interval))
#     # set slice to the first frame of the span
#     viewer.dims.set_point(0, i)
#     # viewer.show(block=True)
#     input("Press Enter to continue...")




In [ ]:
len(single_cells)

In [ ]:
erfqwe. qwef qwef ef 

In [ ]:
sct_operator.remove_empty_contour_sct()

In [ ]:
len(sct_operator.annotate_click_samples["mitosis"])

In [ ]:
# sct_operator.shape_layer.selected_data = [0]
# sc_operator = sct_operator.edit_selected_sc()
# sct_operator.shape_layer.selected_data = [0]
# sc_operator = sct_operator.edit_selected_sc()

In [ ]:
# sc_operator.csn_correct_seg_callback()
# sc_operator.save_seg_callback()
# sc_operator.filter_cells_by_size_callback(min_size=100, max_size=1000)

In [ ]:
# len(sc_operator.shape_layer.selected_data), len(sc_operator.shape_layer.data)

In [ ]:
# sc_operator.resample_contours_callback(40)
# sc_operator.restore_sc_contour_callback()
# sct_operator.clear_sc_opeartors()
# sct_operator.add_new_sc()

In [ ]:
sct_operator.sc_operators = []

In [ ]:
all_scs = sct_operator.get_all_scs()
len(all_scs)

In [ ]:
sct_operator.clear_sc_opeartors()

In [ ]:
sct_operator.edit_selected_sc()

In [ ]:
sct_operator.clear_sc_opeartors()

In [ ]:
# sct_operator.viewer.layers.remove(sct_operator.sc_operators[0].shape_layer)

Update SctOperator's shape layer by an updated single cell

In [ ]:
sct_operator.update_shape_layer_by_sc(single_cells[0])

Get all the single cells from the sct operators

In [ ]:
len(sct_operator.get_all_scs()), len(set(sct_operator.get_all_scs()))

Save annotated single cells

In [ ]:
# sct_operator.restore_shapes_data()

In [ ]:
all_scs = sct_operator.get_all_scs()
SingleCellStatic.write_single_cells_json(all_scs, path="./datasets/test_scs_EBSS_starvation/XY1/corrected_scs.json", \
                                         dataset_dir="./datasets/test_scs_EBSS_starvation/XY1/datasets")

In [ ]:
sct_operator.select_info

Programmatically delete a sample

In [ ]:
sct_operator.annotate_click_samples["mitosis"][-1]

In [ ]:
# for sc in sct_operator.annotate_click_samples["mitosis"][-1]:
#     sc.meta = {}
# sct_operator.annotate_click_samples["mitosis"].pop(-1)

In [ ]:
sct_operator.save_annotations(sample_out_dir="./datasets/test_scs_EBSS_starvation/XY1/annotations", sample_dataset_dir="./datasets/test_scs_EBSS_starvation/XY1/datasets")

In [ ]:
scs = sct_operator.get_all_scs()

In [ ]:
sct_operator.annotate_click_samples

In [ ]:
len(sct_operator.annotate_click_samples["mitosis"])

track

In [ ]:
from livecell_tracker.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)

all_scs = sct_operator.get_all_scs()

all_scs = [sc for sc in all_scs if sc.timeframe != 1]
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

Show subset of a sctc filtered by time_span

In [ ]:
from livecell_tracker.core.sct_operator import create_scts_operator_viewer, create_scs_edit_viewer

scts_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, time_span=(145, 155))

# # If you would like to start from sctc, you can use the following code
# scts_operator = create_scts_operator_viewer(sctc, img_dataset = dic_dataset, time_span=(1, 2))

In [ ]:
# sctc.write_json(path="./EBSS_starvation_24h_xy16_annotation/single_cell_trajectory_collection.json")